In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma2/keras/gemma2_instruct_2b_en/1/config.json
/kaggle/input/gemma2/keras/gemma2_instruct_2b_en/1/tokenizer.json
/kaggle/input/gemma2/keras/gemma2_instruct_2b_en/1/metadata.json
/kaggle/input/gemma2/keras/gemma2_instruct_2b_en/1/model.weights.h5
/kaggle/input/gemma2/keras/gemma2_instruct_2b_en/1/assets/tokenizer/vocabulary.spm
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/variations_df.csv
/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/final_qa_pairs_dataset.csv
/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma/config.json
/kaggle/input/version107_foodiefinder_k

# Adding Memory: ChatState

In [66]:
%%time
import os
os.environ["KERAS_BACKEND"] = "jax" 

import keras
import keras_nlp
from IPython.display import Markdown, display
import textwrap

CPU times: user 278 µs, sys: 6 µs, total: 284 µs
Wall time: 291 µs


In [67]:
%%time
# my custom model
gemma_finetuned = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma")  # gemma_instruct_2b_en

CPU times: user 1min 7s, sys: 1min 18s, total: 2min 26s
Wall time: 1min 55s


In [68]:
def display_chat(prompt, response):
  '''Displays an LLM prompt and response in a pretty way.'''
  prompt = prompt.replace('\n\n','<br><br>')
  prompt = prompt.replace('\n','<br>')
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  response = response.replace('•', '  *')
  response = textwrap.indent(response, '', predicate=lambda _: True)
  response = response.replace('\n\n','<br><br>')
  response = response.replace('\n','<br>')
  response = response.replace("```","")
  formatted_text = "<font size='+1' color='teal'>🤖<blockquote>" + response + "</blockquote></font>"
  return Markdown(formatted_prompt+formatted_text)

## Interaction without memory

In [28]:
template = "Question:\n{question}\n\nAnswer:\n{answer}"

In [29]:
%%time
prompt = template.format(
    question="What are some restaurants that sell good tempura in Japan?",
    answer="",
)
completion = gemma_finetuned.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

<font size='+1' color='brown'>🙋‍♂️<blockquote>Question:
What are some restaurants that sell good tempura in Japan?

Answer:
</blockquote></font><font size='+1' color='teal'>🤖

> Tempura Iseya in Ueno, Tokyo, is a well-known tempura restaurant where diners can watch the chef prepare freshly fried tempura right before their eyes. The light batter enhances the natural flavors of fresh, seasonal ingredients, including seafood and vegetables. No reservations are required, but it can get busy. Be sure to enjoy the tempura while it's hot, adjusting the flavor with dipping sauce or salt according to your preference.
</font>

In [31]:
%%time
prompt = template.format(
    question="What type of food did I just ask about?",
    answer="",
)
completion = gemma_finetuned.generate(prompt, max_length=1024)
response = completion.replace(prompt, "")
display_chat(prompt, response)

CPU times: user 3min 35s, sys: 1.6 s, total: 3min 36s
Wall time: 1min 13s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Question:
What type of food did I just ask about?

Answer:
</blockquote></font><font size='+1' color='teal'>🤖

> You are not provided with details about the restaurant you are asking about.
</font>

## Interaction with memory

In [69]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """
  def __init__(self, model):
    """
    Initializes the chat state.
    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma2_instruct_2b_en") # /kaggle/input/version107_foodiefinder_kagglex_v2/keras/version107/1/foodie_finder_gemma
    self.history = []
  def add_to_history_as_user(self, message):
    """
    Adds a user message to the history with start/end turn markers.
    """
    self.history.append(message)
  def add_to_history_as_model(self, message):
    """
    Adds a model response to the history with start/end turn markers.
    """
    # remove new lines
    message = message.replace("\n"," ")
    self.history.append(message )
  def get_history(self):
    """
    Returns the entire chat history as a single string.
    """
    return "".join([*self.history])
  def get_history_blurb(self):
    """
    Returns what to insert into the current prompt
    """
    if len(self.history)==0:
      return ""
    else:
      return \
f"""\n\nUse the following chat history context to respond to the instruction below:\n"""\
f"""{self.get_history()}"""

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history()
    return prompt
  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.
    Args:
        message: The user's message.
    Returns:
        The model's response.
    """
    # Step 2: Fake retrieving context from Chroma
    #chroma_context = "Some people are allergic to aspirin. "
    # Step 3: Fake retrieving web search context
    #web_context = "Many drugs have harmful interactions if taken together. "
    # Step 4: Construct prompt with both Chroma and web search contexts
    prompt = self.get_full_prompt()
    full_prompt = \
f"""You are an AI assistant that responds to instructions about food"""\
f"""{self.get_history_blurb()}\n"""\
f"""Instruction: {message}"""\
f"""Response:"""
    # GW for debugging - print("--->\n" + full_prompt + "<--")
    self.add_to_history_as_user(message)
    # Generate response with full prompt
    response = self.model.generate(full_prompt, max_length=1024)
    # GW for debugging - print("--->\n" + response + "<--")
    result = response.replace(full_prompt, "")  # Extract only the new response
    # Add the result to chat history
    self.add_to_history_as_model(result)

    return result

In [70]:
chat = ChatState(gemma_finetuned)

In [71]:
%%time
# Can I find tempura in Japan? 
prompt = "Can I find tempura in Japan?"
response = chat.send_message(prompt)
display_chat(prompt, response)

CPU times: user 5min 7s, sys: 1.96 s, total: 5min 9s
Wall time: 2min 8s


<font size='+1' color='brown'>🙋‍♂️<blockquote>Can I find tempura in Japan?</blockquote></font><font size='+1' color='teal'>🤖<blockquote>While not as widely served as in past years, tempura is still found in many restaurants in Japan, with Kyoto and Osaka being the most prolific regions with multiple restaurants in each region.</blockquote></font>

In [72]:
%%time
prompt = "What food are we discussing about?"
response = chat.send_message(prompt)
display_chat(prompt, response)

CPU times: user 3min 36s, sys: 1.63 s, total: 3min 37s
Wall time: 1min 14s


<font size='+1' color='brown'>🙋‍♂️<blockquote>What food are we discussing about?</blockquote></font><font size='+1' color='teal'>🤖<blockquote> The text "Tempura" was not found in the provided context.</blockquote></font>